# Uppgift 2

### Antal län i datasetet

In [13]:
import pandas as pd

excel_file_path = "Labb-data/Folkhalsomyndigheten_Covid19_Vaccine.xlsx"

df = pd.read_excel(excel_file_path, sheet_name=14)

antal_lan = df["Län_namn"].nunique()

print(f"Antal län representerade i datasetet: {antal_lan}")
df

Antal län representerade i datasetet: 21


,Län,Län_namn,Kommun,Kommun_namn,Ålder,Befolkning,Antal minst 1 dos,Antal minst 2 doser,Antal 3 doser,Antal 4 doser,Andel minst 1 dos,Andel minst 2 doser,Andel 3 doser,Andel 4 doser
0,1,Stockholms län,114,Upplands Väsby,12-15,2422,1206,1046,NaN,NaN,0.497936,0.431874,NaN,NaN
1,1,Stockholms län,114,Upplands Väsby,16-17,1203,839,755,NaN,NaN,0.697423,0.627598,NaN,NaN
2,1,Stockholms län,114,Upplands Väsby,18-29,6692,4887,4469,1959.0,NaN,0.730275,0.667812,0.292738,NaN
3,1,Stockholms län,114,Upplands Väsby,30-39,7332,5542,5240,2878.0,NaN,0.755865,0.714675,0.392526,NaN
4,1,Stockholms län,114,Upplands Väsby,40-49,6946,5592,5429,3719.0,NaN,0.805068,0.781601,0.535416,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2895,25,Norrbottens län,2584,Kiruna,50-59,3079,2878,2860,2482.0,NaN,0.934719,0.928873,0.806106,NaN
2896,25,Norrbottens län,2584,Kiruna,60-69,2781,2648,2633,2434.0,NaN,0.952175,0.946782,0.875225,NaN
2897,25,Norrbottens län,2584,Kiruna,70-79,2194,2115,2108,2034.0,1784.0,0.963993,0.960802,0.927074,0.813127
2898,25,Norrbottens län,2584,Kiruna,80-89,1280,1256,1253,1220.0,1091.0,0.981250,0.978906,0.953125,0.852344


### Antal kommuner i datsetet 

In [14]:
antal_kommuner = df["Kommun_namn"].nunique()
print(f"Antal kommuner representerade i datasetet: {antal_kommuner}")


Antal kommuner representerade i datasetet: 290


### Antal befolkning

In [22]:
antal_befolkning = df["Befolkning"].sum()
print(f"Antal befolkning som är med i datasetet: {antal_befolkning} personer")

Total population in dataset: 9092790 personer


### Antal barn med i datasetet
Antal barn i sverige mellan 12-18 år är runt 779 000, enligt datsetet så är 745 370 av dem vaccinerade (källa: scb)

In [26]:
def extract_lower_bound(age_range):
    if "eller" in age_range:
        return 90  
    return int(age_range.split("-")[0])

df["Lägsta_ålder"] = df["Ålder"].apply(extract_lower_bound)


filtered_df = df[df["Lägsta_ålder"] < 18]

total_population_under_18 = filtered_df["Befolkning"].sum()

print("Antal befolkning under 18:", total_population_under_18)


Antal befolkning under 18: 745370


### Åldersföredling av svergies befolkning

In [8]:
import pandas as pd
import plotly.express as px

df = pd.read_excel(excel_file_path, sheet_name=14)

fig = px.bar(df, x="Ålder", y="Befolkning", color="Ålder", labels={"Befolkning": "Antal"})

fig
# fig.write_html("uppgift2e.html")

### Stapeldiagram för Antal vaccinerade i olika län

In [ ]:
import pandas as pd
import plotly.express as px

df = pd.read_excel(excel_file_path, sheet_name=14)


df_doses = pd.DataFrame({
    "County": df["Län"],
    "Dose 1": df["Antal minst 1 dos"],
    "Dose 2": df["Antal minst 2 doser"],
    "Dose 3": df["Antal 3 doser"]
})


df_melted = pd.melt(df_doses, id_vars=["County"], var_name="Dose", value_name="TotalVaccinated")


fig = px.bar(df_melted, x="County", y="TotalVaccinated", color="Dose",
             title="Antal som är vaccinerad med en, två eller tre doser per län",
             labels={"TotalVaccinated": "Antal vaccinerade", "Dose": "Vaccination Doser", "County": "Län"})
fig.show()
# fig.write_html("plotly_barchart_vaccine.html")

### Jämför vaccinerade i Västra götalands län och Stockholms län

In [ ]:
import pandas as pd
import plotly.express as px

df = pd.read_excel(excel_file_path, sheet_name=14)

selected_Counties = ["Västra Götalands län", "Stockholms län"]

df_selected = df[df["Län_namn"].isin(selected_Counties)]

df_doses_selected = pd.DataFrame({
    "County": df["Län_namn"],
    "Dose 1": df["Antal minst 1 dos"],
    "Dose 2": df["Antal minst 2 doser"],
    "Dose 3": df["Antal 3 doser"],
    "Dose 4": df["Antal 4 doser"]  
})

df_melted_selected = pd.melt(df_doses_selected, id_vars=["County"], var_name="Dose", value_name="TotalVaccinated")

fig = px.bar(df_melted_selected, x="County", y="TotalVaccinated", color="Dose",
             title="Antal Vaccinationer i Västra Götalands län och Stockholms län",
             labels={"TotalVaccinated": "Antal vaccinerade", "Dose": "Vaccination doser", "County": "Län"})



fig.show()
# fig.write_html("plotly_County_vaccine.html")